In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')
milton = gutenberg.raw('milton-paradise.txt')
sense = gutenberg.raw('austen-sense.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
sense = re.sub(r'Chapter \d+', '', sense)
alice = re.sub(r'CHAPTER .*', '', alice)
milton = re.sub(r'Book \d+', '', milton)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
sense = text_cleaner(sense[:int(len(sense)/10)])
milton = text_cleaner(milton[:int(len(milton)/10)])

In [3]:
# Print the first x characters of text
print('\nRaw:\n', milton[0:500])


Raw:
 Book I Of Man's first disobedience, and the fruit Of that forbidden tree whose mortal taste Brought death into the World, and all our woe, With loss of Eden, till one greater Man Restore us, and regain the blissful seat, Sing, Heavenly Muse, that, on the secret top Of Oreb, or of Sinai, didst inspire That shepherd who first taught the chosen seed In the beginning how the heavens and earth Rose out of Chaos: or, if Sion hill Delight thee more, and Siloa's brook that flowed Fast by the oracle of G


In [4]:
nlp = spacy.load('en')

# All the processing work is done here, so it may take a while.
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)
sense_doc = nlp(sense)
milton_doc = nlp(milton)

In [7]:
# Extract the first ten entities.
entities = list(alice_doc.ents)[0:10]
for entity in entities:
    print(entity.label_, ' '.join(t.orth_ for t in entity))

PERSON Alice
DATE the hot day
PERSON Alice
PERSON Rabbit
PERSON Rabbit
PERSON Alice
PERSON Alice
PERSON Alice
ORDINAL First
CARDINAL one


In [24]:
# Group into sentences.
alice_sents = [[sent, "Carroll",len([token for token in sent if not token.is_punct])] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen",len([token for token in sent if not token.is_punct])] for sent in persuasion_doc.sents]
sense_sents = [[sent, "Austen2",len([token for token in sent if not token.is_punct])] for sent in sense_doc.sents]
milton_sents = [[sent, "Milton",len([token for token in sent if not token.is_punct])] for sent in milton_doc.sents]

# Combine the sentences from the four novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents + sense_sents + milton_sents)
sentences.head()

,0,1,2
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,57
1,"(So, she, was, considering, in, her, own, mind...",Carroll,56
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,27
3,"(Oh, dear, !)",Carroll,2
4,"(Oh, dear, !)",Carroll,2


In [27]:
# Exclude stopwords and punctuation and use lemmas (root words) and limit to 1000 most common words for each text

# Utility function to create a list of the 1000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df['words_per_sentence'] = sentences[2]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)
sensewords = bag_of_words(sense_doc)
miltonwords = bag_of_words(milton_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords + sensewords + miltonwords)

In [28]:
common_words

{'approbation',
 'hasten',
 'eye',
 'advise',
 'alter',
 'separation',
 'discover',
 'difficulty',
 'household',
 'chivalry',
 'Confounded',
 'pomp',
 'singularity',
 'inconvenience',
 "know'st",
 'form',
 'relish',
 'headlong',
 'suddenly',
 'heave',
 'accidentally',
 'pin',
 'ofttime',
 'grief',
 'Gloucester',
 'pop',
 'plainly',
 'FOOT',
 'bloom',
 'transgress',
 'county',
 'humoured',
 'Uppercross',
 'despise',
 'singe',
 'comfortable',
 'subterranean',
 'strive',
 'tone',
 'connect',
 'Walter',
 'frost',
 'extort',
 'cherry',
 'white',
 'reasonably',
 'mistress',
 'seriously',
 'calamity',
 'foam',
 'tall',
 'blossom',
 'revolve',
 'instantly',
 'assurance',
 'especially',
 'pine',
 'White',
 'help',
 'conceive',
 'village',
 'wind',
 'heavenly',
 'noise',
 'skurrie',
 'discharge',
 'truth',
 'stir',
 'grant',
 'Titanian',
 'relate',
 'regulation',
 'thought',
 'troop',
 'age',
 'sign',
 'requisition',
 'high',
 'fit',
 'Mightiest',
 'impenetrable',
 'prudent',
 'alloy',
 'stay',


In [29]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950
Processing row 1000
Processing row 1050
Processing row 1100
Processing row 1150
Processing row 1200
Processing row 1250
Processing row 1300
Processing row 1350


,approbation,hasten,eye,advise,alter,separation,discover,difficulty,household,chivalry,...,November,unheard,handsome,unusual,speedily,revive,size,text_sentence,text_source,words_per_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,57
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,56
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,27
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,2


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression(penalty='l2')

train = lr.fit(X_train, y_train)

print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(825, 2486) (825,)
Training set score: 0.9163636363636364

Test set score: 0.7618181818181818


In this case, adding number of words per sentence, the test set score is greatly improve
The training score declined slightly  - try making the overall set bigger - back to 2000 common words

In [32]:
# Exclude stopwords and punctuation and use lemmas (root words) and limit to 2000 most common words for each text

# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df['words_per_sentence'] = sentences[2]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)
sensewords = bag_of_words(sense_doc)
miltonwords = bag_of_words(milton_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords + sensewords + miltonwords)

In [33]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950
Processing row 1000
Processing row 1050
Processing row 1100
Processing row 1150
Processing row 1200
Processing row 1250
Processing row 1300
Processing row 1350


,approbation,excellence,hasten,underneath,eye,advise,Horonaim,crystal,alter,separation,...,parlor,orgy,organ,speedily,revive,Damascus,size,text_sentence,text_source,words_per_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,57
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,56
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,27
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,2


In [34]:
# Also, adjust test size

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=0)

lr = LogisticRegression(penalty='l2')

train = lr.fit(X_train, y_train)

print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(1100, 3833) (1100,)
Training set score: 0.9281818181818182

Test set score: 0.7563636363636363


This is very similar result to first so additional common words do not seem necessary to produce a substantially similar result